In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import re
import os
import pickle

In [2]:
import urllib.parse as p

In [ ]:
"""
CountryCode
language

1. How to program inPython.
2. Python for beginers
3. Numpy tutorials
5. Python strings
6. Advanced python programming
7. How to solve x

"""

In [ ]:
class YouTubeVideoStats:
    pass

In [22]:
class Query:
    """A query to pass to the search resource."""
    
    def __init__(self, query_string: str):
        self.__query_string = query_string
        
    @property
    def query_string(self):
        return self.__query_string
    
    @query_string.setter
    def query_string(self, query_str: str):
        if not query_str:
            raise ValuError('The query string has to be provided')
        if not isinstance(query_str, str):
            raise TypeError('The query string has to be a string')
        self.__query_string = query_str

        
"""
Returns a collection of search results that match the query parameters specified in 
the API request. By default, a search result set identifies matching video, channel, 
and playlist resources, but you can also configure queries to only retrieve a specific 
type of resource.

{
  "kind": "youtube#searchResult",
  "etag": etag,
  "id": {
    "kind": string,
    "videoId": string,
    "channelId": string,
    "playlistId": string
  },
  "snippet": {
    "publishedAt": datetime,
    "channelId": string,
    "title": string,
    "description": string,
    "thumbnails": {
      (key): {
        "url": string,
        "width": unsigned integer,
        "height": unsigned integer
      }
    },
    "channelTitle": string,
    "liveBroadcastContent": string
  }
}

"""
class YouTubeSearch:
    pass

class VideoSearch(YouTubeSearch):
    pass

In [31]:
#api.constants.YouTubeAPIContants
class YouTubeAPIConstants:
    TOKEN_FILE = 'token.pickle'
    API_SERVICE_NAME = 'youtube'
    API_VERSION = 'v3'
    SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

class Authenticate:
    """Handle the YouTube authentication process."""
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    TOKEN_FILE = YouTubeAPIConstants.TOKEN_FILE
    API_SERVICE_NAME = YouTubeAPIConstants.API_SERVICE_NAME
    API_VERSION = YouTubeAPIConstants.API_VERSION
    SCOPES = YouTubeAPIConstants.SCOPES
    
    def __init__(self, client_secrets_file: str , api_token_path: str = ''):
        """Create the auth object."""
        self.credentials = None
        
        self.__verify_client_secret_file(client_secrets_file)
        self.client_secrets_file = client_secrets_file
        
        if not api_token_path or not os.path.exists(api_token_path):
            self.api_token_path = self.__get_default_api_token_path()
        else:
            self.api_token_path = api_token_path
            
    def __verify_client_secret_file(self, client_secrets_file: str) -> None:
        """Verfy the client secret file."""
        if not client_secrets_file:
            raise ValueError('The clients secret file path has to be provided.')
        if not isinstance(client_secrets_file, str):
            raise TypeError('The clients secret file should be a string.')
        if not os.path.exists(client_secrets_file):
            raise ValueError(f'The path {client_secrets_file} does not exist!')
        
    def __get_default_api_token_path(self):
        """Generate the default api token file location."""
        current_user_home_dir = os.path.expanduser('~')
        api_token_path = os.path.join(current_user_home_dir, self.TOKEN_FILE)
        return api_token_path
    
    def __authenticate_youtube(self):
        """Authenticate the YouTube API."""
        if os.path.exists(self.api_token_path):
            with open(self.api_token_path, "rb") as token:
                self.credentials = pickle.load(token)
        # if there are no (valid) credentials availablle, let the user log in.
        if not self.credentials or not self.credentials.valid:
            if self.credentials and self.credentials.expired and self.credentials.refresh_token:
                self.credentials.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.client_secrets_file, self.SCOPES)
                self.credentials = flow.run_local_server(port=0)
            # save the credentials for the next run
            with open(self.api_token_path, "wb") as token:
                pickle.dump(self.credentials, token)

        return build(self.API_SERVICE_NAME, self.API_VERSION, credentials=self.credentials)
    
    def authenticate(self):
        return self.__authenticate_youtube()
        
class YouTube:
    def __init__(self, credentials_file='', token_path=''):
        self.__auth = Authenticate(credentials_file, token_path)
        self.__youtube_client = None
        
    def authenticate(self, credentials_file='', token_path=''):
        self.__youtube_client = self.__auth.authenticate()
        
    def get_youtube(self):
        return self.__youtube_client
        
    def search_video(self) -> list[str]:
        request = self.__youtube_client.search().list(
            part="id,snippet",
            type='video',
            q="Spider-Man",
            videoDuration='short',
            videoDefinition='high',
            maxResults=1
        )
        # Request execution
        response = request.execute()
        print(response)
        
    def find_video(self, video: str, by: str = 'url'):
        """Find a video by url or id."""
        if by:
            if not isinstance(by, str):
                raise ValueError('by must be a string')
            if by not in ['url', 'id']:
                raise ValueError('by must be either id or url')
        if by == 'url':
            return self.__get_video_id(video)
    
    @staticmethod
    def __get_video_id(video_url: str) -> str:
        """Get vdeo ID from video url"""
        if not video_url:
            raise ValueError('The video_ur has to be provided.')
        if not isinstance(video_url, str):
            raise TypeError('Te video_url has to be a string.')
        if '=' not in video_url:
            url_format = 'https://www.youtube.com/watch?v=Dqdu-FsBk0s'
            raise ValueError('Te video_url should be of the format "{url_format}"')
        video_url = video_url.split('=')[1]
        return video_url

In [ ]:
"""
youtube = YouTube()
youtube.authenticate(credentials_file='', token_path='')
youtube = YouTube(credentials_file='', token_path='')
youtube.search_video(): videoId, videoUrl -> List[YouTubeVideo]
youtube.search_channel(): channelID -> List[YouTubeChannel]
youtube.search_playlist(): playlistId -> List[PlayList]
"""

In [32]:
youtube = YouTube(credentials_file='/home/lyle/Downloads/credentials.json', 
                  token_path='')
youtube.authenticate()

In [33]:
youtube_client = youtube.get_youtube()

In [37]:
def search_video():
    request = youtube_client.search().list(
            part="id,snippet",
            type='video',
            q="Spider-Man",
            videoDuration='short',
            videoDefinition='high',
            maxResults=10
    )
    # Request execution
    response = request.execute()
    return (response)

In [38]:
spider_man = search_video()

In [39]:
spider_man

{'kind': 'youtube#searchListResponse',
 'etag': 'e5_ZI3NMxWDspDvbuuORnXZXTus',
 'nextPageToken': 'CAoQAA',
 'regionCode': 'KE',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 10},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'nnwDqUXMNsScosopUGk7IYC3G6c',
   'id': {'kind': 'youtube#video', 'videoId': 'uKYV2qjYIS0'},
   'snippet': {'publishedAt': '2023-02-20T17:43:03Z',
    'channelId': 'UC-pnSdXRsSi44apSO5RXjlw',
    'title': 'Spooder-Man: Across The Spooder-Verse Trailer',
    'description': 'SPOODERMAN MERCH : https://shop.laughoverlife.com/ In this newest official unofficial movie trailer featuring our special ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/uKYV2qjYIS0/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/uKYV2qjYIS0/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/uKYV2qjYIS0/hqdefault.jpg',
      'width': 480,
      'height'

In [21]:
url = 'https://www.youtube.com/watch?v=Dqdu-FsBk0s'
youtube.find_video(url)

url


'Dqdu-FsBk0s'

In [ ]:
"""
I want to be able to fetch:
1. Get a single youtube video
2. Get a playlist
3. Get a single channel
4. Get comments
5. Get video stats
6. Get channel stats
"""

In [ ]:
"""
The items that can be fetched frm the YouTube API arecalled Resources.
The Resources include:
1. Activities
2. Captions
3. Channels
4. ChannelSectons
5. Comments
6. CommentThreads

Each Resource supports various methods of most importance is the list method. They also
have a representation, which is the response returned by the API. An example video 
representation is:


"""

In [ ]:
"""
Resources: 
---------
YouTube's entities from which you can extract information.

Methods: 
-------
Functions which can be applied to the resources in order to extract their properties 
or modify them.

List methods: 
------------
This is the method of our interest since it can be used to retrieve collections of 
resources and then extract information about each one of them.

Overview section: 
----------------
Where the resource is explained. The most interesting part here is the Resource 
representation subsection. It will be useful since from there you will select the 
particular properties you want to retrieve from the resources you request.

list section: 
------------
Where the list method for each resource is explained.
— In the Parameters subsection you can see the required and optional parameters that 
can be used in order to customize the queries.
— In the Response subsection you can see the Response representation which includes 
the properties the query response has. It is important because you can also retrieve 
or ignore attributes of the responses.
"""

In [ ]:
"""
Video Resource Response
-----------------------

{
  "kind": "youtube#videoListResponse",
  "etag": etag,
  "nextPageToken": string,
  "prevPageToken": string,
  "pageInfo": {
    "totalResults": integer,
    "resultsPerPage": integer
  },
  "items": [
    video Resource
  ]
}

Errors
------
badRequest (400) videoChartNotFound 
The requested video chart is not supported or is not available.

forbidden (403) forbidden
The request is not properly authorized to access video file or processing information. 
Note that the fileDetails, processingDetails, and suggestions parts are only available 
to that video's owner.

forbidden (403) forbidden
The request cannot access user rating information. This error may occur because the 
request is not properly authorized to use the myRating parameter.

notFound (404) videoNotFound
The video that you are trying to retrieve cannot be found. Check the value of the 
request's id parameter to ensure that it is correct.

"""

In [3]:
query = Query("Python")
query.query_string

'Python'

In [ ]:
class YouTubeVideo:
    """A YouTube Video."""
    pass

In [ ]:
class YouTubeChannel:
    pass

In [ ]:
class YouTubePlayList

In [ ]:
class YouTubeCommentThread:
    pass

In [ ]:
class Stats:
    pass

class YouTubeVideoStats:
    pass

class YouTubeChannelStats:
    pass

In [ ]:
class YouTubeComment:
    pass

In [38]:
def get_video_details(youtube, **kwargs):
    return youtube.videos().list(
        part="snippet,contentDetails,statistics",
        **kwargs
    ).execute()

In [39]:
def print_video_infos(video_response):
    items = video_response.get("items")[0]
    # get the snippet, statistics & content details from the video response
    snippet         = items["snippet"]
    statistics      = items["statistics"]
    content_details = items["contentDetails"]
    # get infos from the snippet
    channel_title = snippet["channelTitle"]
    title         = snippet["title"]
    description   = snippet["description"]
    publish_time  = snippet["publishedAt"]
    # get stats infos
    comment_count = statistics["commentCount"]
    like_count    = statistics["likeCount"]
    view_count    = statistics["viewCount"]
    # get duration from content details
    duration = content_details["duration"]
    # duration in the form of something like 'PT5H50M15S'
    # parsing it to be something like '5:50:15'
    parsed_duration = re.search(f"PT(\d+H)?(\d+M)?(\d+S)", duration).groups()
    duration_str = ""
    for d in parsed_duration:
        if d:
            duration_str += f"{d[:-1]}:"
    duration_str = duration_str.strip(":")
    print(f"""\
    Title: {title}
    Description: {description}
    Channel Title: {channel_title}
    Publish time: {publish_time}
    Duration: {duration_str}
    Number of comments: {comment_count}
    Number of likes: {like_count}
    Number of views: {view_count}
    """)

In [40]:
def get_video_id_by_url(url):
    """
    Return the Video ID from the video `url`
    """
    # split URL parts
    parsed_url = p.urlparse(url)
    # get the video ID by parsing the query of the URL
    video_id = p.parse_qs(parsed_url.query).get("v")
    if video_id:
        return video_id[0]
    else:
        raise Exception(f"Wasn't able to parse video URL: {url}")

In [41]:
video_url = "https://www.youtube.com/watch?v=jNQXAC9IVRw&ab_channel=jawed"
# parse video ID from URL
video_id = get_video_id_by_url(video_url)
# make API call to get video info
response = get_video_details(youtube, id=video_id)
# print extracted video infos
print_video_infos(response)

    Title: Me at the zoo
    Description: 
    Channel Title: jawed
    Publish time: 2005-04-24T03:31:52Z
    Duration: 19
    Number of comments: 11269824
    Number of likes: 13253111
    Number of views: 257732508
    
